# Capstone Project - Food Classifier

## Overview

The aim of this Capstone Project is to use deep learning to classify images of food. The notebook will include all implementation stages of the project including, data processing, training and building the model and finally an analysis of the results. 

## Software Requirements

The project requires an installation of Anaconda with Python 2. The main libraries use will be Keras with a Tensorflow backend, panda and numpy. 

## Data Processing

The project uses the UECFOOD256 from an academic insitute in Japan, the dataset contains 31,651 images in 256 folders with each folder being a particular category of the food.

### Image Resize

The first step is to resize all the images in order to standardize the input dimensions for the convolutional neural network. 

In [26]:
from os import listdir
import os.path
import pandas as pd
from sklearn.datasets import load_files    
from sklearn.model_selection import train_test_split
import numpy as np

path = "D:\Workspace\deeplearning-foodclassifier\UECFOOD256"

#for item in sorted(listdir(path)):
    #dir = path + '/' + item
    #if isdir(dir):        
        #print 'LABEL ' + item
        #print len(listdir(dir))
        
categories = 'D:\Workspace\deeplearning-foodclassifier\labels.txt'     
labels = pd.read_csv(categories, sep=",", header=0)



In [36]:
%%time
total = 0
for item in listdir(path):
    image_folder = os.path.join(path, item)
    if isdir(image_folder):        
        images = [file for file in os.listdir(image_folder) if file.endswith('.jpg')]
        print 'Folder ' + item + ' has ' + str(len(images)) + ' images'
        total = total + len(images)
        
print 'total files ' + str(total)
    
image_folder = os.path.join(path, '1')
files = [file for file in os.listdir(image_folder) if file.endswith('.jpg')]

#print labels
#print labels.iloc[2]["name"]

X = np.vstack(files)
y = np.vstack(['rice'] * len(files))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

print str(len(X_train))
print str(len(y_train))
print str(len(X_val))
print str(len(y_val))
print str(len(X_test))
print str(len(y_test))

# Make the code work for one label (rice)
# Next step is to copy the images to 3 folders


Folder 1 has 620 images
Folder 10 has 136 images
Folder 100 has 104 images
Folder 101 has 100 images
Folder 102 has 108 images
Folder 103 has 103 images
Folder 104 has 104 images
Folder 105 has 107 images
Folder 106 has 105 images
Folder 107 has 101 images
Folder 108 has 101 images
Folder 109 has 101 images
Folder 11 has 112 images
Folder 110 has 105 images
Folder 111 has 111 images
Folder 112 has 102 images
Folder 113 has 104 images
Folder 114 has 105 images
Folder 115 has 106 images
Folder 116 has 101 images
Folder 117 has 102 images
Folder 118 has 102 images
Folder 119 has 112 images
Folder 12 has 218 images
Folder 120 has 103 images
Folder 121 has 100 images
Folder 122 has 113 images
Folder 123 has 114 images
Folder 124 has 108 images
Folder 125 has 117 images
Folder 126 has 102 images
Folder 127 has 118 images
Folder 128 has 104 images
Folder 129 has 114 images
Folder 13 has 120 images
Folder 130 has 103 images
Folder 131 has 111 images
Folder 132 has 105 images
Folder 133 has 116